In [1]:
import numpy as np
import random, math
from os import sys
from nltk import *

In [2]:
corpus = open("./data/text8", "r").read().split(" ")[1:]
vocabulary = open("./data/vocab.txt", "r").read().split("\n")[:-1]

In [3]:
freq = FreqDist(corpus).most_common(10000)
for w in freq:
    if not w[0] in vocabulary:
        vocabulary.append(w[0])

In [4]:
dictionary ={}
word_list = vocabulary + ["<unk>"]
count = 0
for w in word_list:
    dictionary[w] = count
    count += 1

In [5]:
window_size = 10
dimension = 300
learning_rate = 0.1
vocabulary_size = len(vocabulary) + 1
x_max = 700
alpha = 0.75

In [6]:
words = [-1 for i in xrange(len(corpus))]
word_count = [0 for i in xrange(vocabulary_size)]
i = 0
for w in corpus:
    if w in dictionary:
        words[i] = dictionary[w]
    else:
        words[i] = vocabulary_size - 1
    word_count[words[i]] += 1
    i += 1

In [7]:
cooccur = [[0 for i in xrange(vocabulary_size)] for j in xrange(vocabulary_size)]
for i in xrange(len(words)):
    for y in words[i-window_size+1:i]:
        x = words[i]
        cooccur[x][y] += 1
        cooccur[y][x] += 1

In [8]:
def f(x):
    if x > x_max: return 1
    else: return (float(x)/x_max)**alpha

In [9]:
def compute_loss(W, b):
    loss = 0
    for i in xrange(vocabulary_size):
        for j in xrange(i+1, vocabulary_size):
            loss += f(cooccur[i][j])* (np.dot(W[i], W[j]) + b[i] + b[j] - np.log(1 + cooccur[i][j])) ** 2
    return loss

In [10]:
W = (np.random.rand(vocabulary_size, dimension) - 0.5) / float(dimension + 1)
b = (np.random.rand(vocabulary_size) - 0.5) / float(dimension + 1)

In [11]:
for epoch in xrange(10):
    word_shown = [0 for i in xrange(vocabulary_size)]
    for i in xrange(len(words)):
        x = words[i]
        learning_rate = max(0.1 - word_shown[x]/float(word_count[x]), 0.0001)
        word_shown[x] += 1
        if (x == vocabulary_size -1): continue
        for y in words[i-window_size+1:i]:
            if (y == vocabulary_size - 1): continue
            r = np.dot(W[x], W[y]) + b[x] + b[y] - np.log(1 + cooccur[x][y])
            p = learning_rate * f(cooccur[x][y]) * r
            b[x] -= p
            b[y] -= p
            W_x_ = W[x].copy()
            W[x] -= p * W[y]
            W[y] -= p * W_x_
        if (i % 10000 == 0):
            sys.stdout.flush()
            opt = "\r\rtraining epoch "+str(epoch)+" "+str(i+1)+"/"+str(len(words))+" "
            #opt += "loss = "+str(compute_loss(W, b))+"\n"
            sys.stdout.write(opt)

training epoch 9 17000001/17005207 

In [14]:
file = open("vectors.txt", "w")
for i in xrange(4894):
    print >> file, word_list[i], " ".join([str(w) for w in W[i]])
file.close()

In [13]:
#sixers run1
#sixers run0 is word2vec without window random